In [18]:
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re
import time
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/49.0.2623.112 Safari/537.36",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Connection": "close",
    "Accept-Language": "zh-CN,zh;q=0.9"}
session = requests.Session()

def get_CarInfo(url):
    '''
    获取每辆二手车HTML中的车辆信息
    :param url: 手机端二手车URL
    :return: 该二手车的重要信息
    '''
    # 车辆手机端与PC端链接URL（手机端主要获取VIN及其他重要信息，PC端获取外观颜色）
    mobile_url = url
    pc_url = url.replace('https://m.che168.com', 'https://www.che168.com')
    # 一直访问单辆二手车信息网页，直到访问成功（最多10次）
    get_time = 0
    while get_time < 10:
        try:
            html = session.get(url, headers=headers, timeout=30)
            time.sleep(3)
            pc_html = session.get(pc_url, headers=headers,timeout=30)
            html.close()
            pc_html.close()
        except:
            get_time += 1
            time.sleep(10)
        else:
            bsObj = BeautifulSoup(html.text, 'lxml')
            pc_bsObj = BeautifulSoup(pc_html.content.decode('gb18030'), 'html.parser')
            script = bsObj.find('body').find('script', {'type': 'text/javascript'}).get_text()
            script = script.strip()
            get_time = 10

    # title
    title = bsObj.find('h5', {'class': 'car-title two-line'}).get_text()
    # infoid
    infoid = re.search(re.compile(r"infoId = parseInt\('[\d]*'"), script).group()
    infoid = re.search(re.compile(r"'[\d]*'"), infoid).group()[1:-1]
    # 二手信息公布时间
    publicdate = re.search(re.compile(r"publicdate: '[\d-]*'"), script).group()
    publicdate = re.search(re.compile(r"'[\d-]*'"), publicdate).group()[1:-1]
    # 地区-省份/直辖市 and 城市 id
    # location = bsObj.find('meta', {'name': 'location'})['content']
    # location = re.search(re.compile(r"province=\S*;"), location).group()
    p_id = re.search(re.compile(r"pid: '\d*'"), script).group()
    p_id = re.search(re.compile(r"'\d*'"), p_id).group()[1:-1]
    c_id = re.search(re.compile(r"cid: '\d*'"), script).group()
    c_id = re.search(re.compile(r"'\d*'"), c_id).group()[1:-1]
    # 品牌id
    brandid = re.search(re.compile(r"brandid: '\d*'"), script).group()
    brandid = re.search(re.compile(r"'\d*'"), brandid).group()[1:-1]
    # 车系id
    seriesid = re.search(re.compile(r"seriesid: '\d*'"), script).group()
    seriesid = re.search(re.compile(r"'\d*'"), seriesid).group()[1:-1]
    # 车型id
    specid = re.search(re.compile(r"specid: '\d*'"), script).group()
    specid = re.search(re.compile(r"'\d*'"), specid).group()[1:-1]
    # 二手价格
    price = re.search(re.compile(r"price: '[0-9\.]*'"), script).group()
    price = re.search(re.compile(r"'[0-9\.]*'"), price).group()[1:-1]
    # 里程/万公里
    mileage = re.search(re.compile(r"mileage: '[0-9\.]*'"), script).group()
    mileage = re.search(re.compile(r"'[0-9\.]*'"), mileage).group()[1:-1]
    # 车龄
    carAge = re.search(re.compile(r"carAge: '[0-9\.]*'"), script).group()
    carAge = re.search(re.compile(r"'[0-9\.]*'"), carAge).group()[1:-1]
    # 车辆档案
    base_data = bsObj.find('ul', {'class': 'base-data'}).find_all('li')
    # 车辆档案-首次上牌
    registedate = base_data[0].find('p', {'class': 'item-status'}).get_text()
    # 车辆档案-查询准迁地-国几
    how_much_guo = base_data[2].find('p', {'class': 'item-status'}).get_text()
    # 车辆档案-排量、变速箱
    power = base_data[3].find('p', {'class': 'item-status'}).get_text()
    trans = base_data[4].find('p', {'class': 'item-status'}).get_text()
    # 过户次数
    guo_hu = base_data[5].find('p', {'class': 'item-status'}).get_text()
    # 车辆档案-牌照地
    license_plate_area = base_data[6].find('p', {'class': 'item-status'}).get_text()
    vincode = re.search(re.compile(r"vincode: '\w*'"), script).group()
    vincode = re.search(re.compile(r"'\w*'"), vincode).group()[1:-1]
    # 车身颜色
    if ("/dealer/" in url) or ("/personal/" in url):
        color = pc_bsObj.find(text='车身颜色').parent.parent.get_text()
        color = color.replace('车身颜色', '')
    elif "/lianmeng/" in url:
        color = pc_bsObj.find(text='颜　　色：').parent.parent.get_text()
        color = color.replace('颜　　色：', '').strip()

    CarInfo_list = [infoid, mobile_url, publicdate, title, p_id, c_id, brandid, seriesid, specid, vincode,
                    price, mileage, carAge, trans, power, registedate, guo_hu, how_much_guo, color,
                    license_plate_area, html.text]

        #print(url, '未正确获取二手车信息！')
        #CarInfo_list = ["-", mobile_url, "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-",
         #               "-", "-", "-", "-", html.text]
    return CarInfo_list

def get_page_car_url(address, brand, page):
    url = 'https://www.che168.com/%s/%s/a0_0msdgscncgpi1ltocsp%dex/' % (address, brand, page)
    car_url_list = []
    html = urlopen(url, timeout=30)
    bsObj = BeautifulSoup(html, 'lxml')
    li = bsObj.find_all('li', {'class': 'cards-li list-photo-li'})
    for i in li:
        a_tag = i.find('a')
        a_tag_href = a_tag['href']
        car_url_list.append(a_tag_href)
    return car_url_list
def get_page_car_url_new(address, brand, page):
    url = 'https://www.che168.com/%s/%s/a0_0msdgscncgpi1ltocsp%dex/' % (address, brand, page)
    car_url_list = []
    html = session.get(url, headers=headers, timeout=30)
    bsObj = BeautifulSoup(html.content, 'lxml')
    li = bsObj.find_all('li', {'class': 'cards-li list-photo-li'})
    for i in li:
        a_tag = i.find('a')
        a_tag_href = a_tag['href']
        car_url_list.append(a_tag_href)
    return car_url_list


url = 'https://m.che168.com/dealer/345873/35192037.html'
lists = get_CarInfo(url)
print(lists[:-1])
'''
url = 'https://www.che168.com/dealer/341820/35240942.html?pvareaid=100519&userpid=440000&usercid=0#pos=43#page=2#rtype=10#isrecom=0#filter=38aa0_0a0_0a0_0#module=10#refreshid=0#recomid=0#queryid=1575458995$11$576f1ea4-36e7-4caa-b5b6-99b96759af2b$43299#cartype=10'
url = 'https://www.che168.com/dealer/341820/35240942.html?usercid=0#pos=43#page=2#rtype=10#isrecom=0#filter=38aa0_0a0_0a0_0#module=10#refreshid=0#recomid=0#queryid=1575458995$11$576f1ea4-36e7-4caa-b5b6-99b96759af2b$43299#cartype=10'
r = session.get(url,headers=headers,timeout=30)
bsObj = BeautifulSoup(r.content, 'lxml')
print(bsObj)
'''

['35192037', 'https://m.che168.com/dealer/345873/35192037.html', '2019-11-02', '凯越 2008款 1.6LE-MT', '450000', '450900', '38', '875', '3817', 'LSGJA52UX8H149306', '2.00', '10', '12', '手动', '1.6L', '2008-09', '2次', '国V', '黑色', '玉林']


"\nurl = 'https://www.che168.com/dealer/341820/35240942.html?pvareaid=100519&userpid=440000&usercid=0#pos=43#page=2#rtype=10#isrecom=0#filter=38aa0_0a0_0a0_0#module=10#refreshid=0#recomid=0#queryid=1575458995$11$576f1ea4-36e7-4caa-b5b6-99b96759af2b$43299#cartype=10'\nurl = 'https://www.che168.com/dealer/341820/35240942.html?usercid=0#pos=43#page=2#rtype=10#isrecom=0#filter=38aa0_0a0_0a0_0#module=10#refreshid=0#recomid=0#queryid=1575458995$11$576f1ea4-36e7-4caa-b5b6-99b96759af2b$43299#cartype=10'\nr = session.get(url,headers=headers,timeout=30)\nbsObj = BeautifulSoup(r.content, 'lxml')\nprint(bsObj)\n"

In [ ]:
# coding=utf-8
'''原有数据添加info_ID'''
import pymysql

def add_info_id():
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='second_hand_car_info', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE second_hand_car_info")
    sql = '''
        SELECT mobile_url FROM `second_hand_car`CONVERT('123',SIGNED);
    '''
    sql = '''
        SELECT CONVERT(REPLACE(SUBSTRING_INDEX(mobile_url,'/',-1),'.html',''),SIGNED)  FROM `second_hand_car` LIMIT 5
    '''
    cur.execute(sql)
    result = cur.fetchall()
    cur.close()
    conn.close()
    return result

def update_registedate():
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='second_hand_car_info', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE second_hand_car_info")
    
    sql = '''
        UPDATE `second_hand_car`
        SET registedate = REPLACE(REPLACE(registedate,'年','-'),'月','')
    '''
    cur.execute(sql)
    conn.commit()
    cur.close()
    conn.close()
    
def update_trans_and_power():
    conn = pymysql.connect(host='127.0.0.1', user='root', passwd='123456', db='second_hand_car_info', charset='utf8')
    cur = conn.cursor()
    cur.execute("USE second_hand_car_info")
    sql = '''
        SELECT TransAndPower,SUBSTRING_INDEX(TransAndPower,'/',1),SUBSTRING_INDEX(TransAndPower,'/',-1)
        FROM second_hand_car
        LIMIT 100
    '''
    sql = '''
        UPDATE `second_hand_car`
        SET trans = SUBSTRING_INDEX(TransAndPower,'/',1),
            power = SUBSTRING_INDEX(TransAndPower,'/',-1)
        WHERE TransAndPower NOT IN ('-')
    '''
    cur.execute(sql)
    conn.commit()
    cur.close()
    conn.close()

    

update_trans_and_power()
print('GAME OVER！')





In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://m.che168.com/personal/del_33428056.html'

r = requests.get(url)


# bsObj = BeautifulSoup(r.content.decode('gb18030'),'html.parser')
# bsObj = BeautifulSoup(r.content,'lxml')
bsObj = BeautifulSoup(r.text, 'lxml')
a = bsObj.find('div', {'class': 'wrong'}).get_text()
print(a)



In [ ]:
a = [1,3,4]
b = None
a.append(b)
print(a)